In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd
import string
from string import digits
import re
import numpy as np


2023-12-05 13:02:54.150298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

callbacks_list = [
                  EarlyStopping(monitor="val_loss", patience=3),
                  ModelCheckpoint("nmt_model_checkpoint",save_best_only=True),
                  TensorBoard(log_dir="/tensorboard_files")
                  ]

In [3]:
# Using positional encoding to re-inject order information

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [4]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)  # Query: inputs, Value: inputs, Keys: Same as Values by default

        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)  # LayerNormalization + Residual connection

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [5]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        # 2 MultiHead Attention layers - one for self attention and one for cross attention
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True #ensures that the layer will propagate its input mask to its outputs;

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1])) # sequence_length == input_shape[1]
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
              tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask_=None): # two inputs: decoder i/p and encoder o/p
        causal_mask = self.get_causal_attention_mask(inputs)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs
        )

        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [6]:
embed_dim = 512
dense_dim = 2048
num_heads = 8
vocab_size = 20000
sequence_length = 33

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="hindi")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs,mask_=None)

x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 hindi (InputLayer)          [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 512)            1025689   ['english[0][0]']             
 tionalEmbedding)                                         6                                       
                                                                                                  
 positional_embedding_1 (Po  (None, None, 512)            1025689   ['hindi[0][0]']           

## Preparing the data

In [7]:
# df = pd.read_csv("Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
import pandas as pd
url='https://drive.google.com/file/d/1EDHFfGBfuYza19mGdTaIpmcscKP3x-48/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

In [8]:
df.shape

(127607, 3)

In [9]:
df.dropna(inplace=True)

In [10]:
def clean_dataset_rows(df):
    df['english_sentence'] = df['english_sentence'].apply(lambda x: x.lower())
    df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.lower())

    exclude = set(string.punctuation)
    df['english_sentence'] = df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

    remove_digits = str.maketrans('', '', digits)
    df['english_sentence'] = df['english_sentence'].apply(lambda x: x.translate(remove_digits))
    df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
    df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

    df['english_sentence'] = df['english_sentence'].apply(lambda x: x.strip())
    df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.strip())
    
    df['length_eng_sentence']=df['english_sentence'].apply(lambda x:len(x.split(" ")))
    df['length_hin_sentence']=df['hindi_sentence'].apply(lambda x:len(x.split(" ")))
    return df

In [11]:
df = clean_dataset_rows(df)

In [12]:
df=df[df['source']=='ted'] # other sources have higher sequence length ~ 200-400


In [15]:
text_pairs = []
for _, row in df.iterrows():
    english = row['english_sentence']
    hindi = row['hindi_sentence']
    hindi = "[start] " + hindi + " [end]"
    text_pairs.append((english, hindi))

import random
print(random.choice(text_pairs))
print(f"no. of pairs: {len(text_pairs)}")

('and instead of bringing them to the police station', '[start] उन्हें पुलिस स्टेशन ले जाने के बजाय [end]')
no. of pairs: 39881


In [16]:
#splitting data
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [14]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [17]:
# Vectorizing the English and Hindi text pairs

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

# Custom standardization function for hindi
def custom_standardization(input_string):
    # lowercase = tf.strings.lower(input_string)
    # Replace elements of input matching regex pattern with rewrite.
    return tf.strings.regex_replace(input_string, f"[{re.escape(strip_chars)}]", "")


source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_hindi_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_hindi_texts)


In [20]:

batch_size = 64

def format_dataset(eng, hindi):
    eng = source_vectorization(eng)
    hindi = target_vectorization(hindi)
    return ({
        "english": eng,           # encoder input
        "hindi": hindi[:, :-1],   # decoder input - take all batches, and length 0 to -1 (one less than input sentence)
    }, hindi[:, 1:])              # decoder ouput - all batches, 1 shifted indices

def make_dataset(pairs):
    eng_texts, hindi_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    hindi_texts = list(hindi_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, hindi_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset
    # return dataset.shuffle(2048).prefetch(16).cache() #Use in-memory caching to speed up preprocessing.

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [21]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['hindi'].shape: {inputs['hindi'].shape}")
    print(f"targets.shape: {targets.shape}")
    print(targets[3])
    print(inputs['english'][3])

inputs['english'].shape: (64, 33)
inputs['hindi'].shape: (64, 33)
targets.shape: (64, 33)
tf.Tensor(
[   5 8167  729  483  112    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0], shape=(33,), dtype=int64)
tf.Tensor(
[   3 5262 2537 2148    8  197    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0], shape=(33,), dtype=int64)


## Traning and evaluating the model

In [21]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=25, validation_data=val_ds, callbacks=callbacks_list)

Epoch 1/25
437/437 [==============================] - 188s 417ms/step - loss: 4.9044 - accuracy: 0.2797 - val_loss: 4.9096 - val_accuracy: 0.2789
Epoch 2/25
437/437 [==============================] - 180s 412ms/step - loss: 4.7134 - accuracy: 0.2921 - val_loss: 4.8360 - val_accuracy: 0.2907
Epoch 3/25
437/437 [==============================] - 180s 412ms/step - loss: 4.5586 - accuracy: 0.3069 - val_loss: 4.7561 - val_accuracy: 0.3073
Epoch 4/25
437/437 [==============================] - 180s 412ms/step - loss: 4.3835 - accuracy: 0.3235 - val_loss: 4.6857 - val_accuracy: 0.3150
Epoch 5/25
437/437 [==============================] - 180s 412ms/step - loss: 4.1939 - accuracy: 0.3405 - val_loss: 4.6333 - val_accuracy: 0.3254
Epoch 6/25
437/437 [==============================] - 180s 412ms/step - loss: 3.9724 - accuracy: 0.3595 - val_loss: 4.5585 - val_accuracy: 0.3369
Epoch 7/25
437/437 [==============================] - 180s 412ms/step - loss: 3.6956 - accuracy: 0.3839 - val_loss: 4.4860 -

In [28]:
hindi_vocab = target_vectorization.get_vocabulary()
hindi_index_lookup = dict(zip(range(len(hindi_vocab)), hindi_vocab))
max_decoded_sentence_length = 30

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = hindi_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(4):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
dont believe a word of it
[start] आप में से विश्वास नहीं करते हैं [end]
-
of over  prisoners
[start] ब्रुकलिन के आसपास थे [end]
-
and second of all
[start] और दूसरा समूह [end]
-
and thats true
[start] और यह सच है [end]


### Inference from saved model :

In [ ]:
with keras.utils.custom_object_scope({'PositionalEmbedding': PositionalEmbedding,
                                     'TransformerEncoder':TransformerEncoder,
                                     'TransformerDecoder':TransformerDecoder}):
    loaded_model = keras.models.load_model("NMT_model.h5")


def get_predictions(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = loaded_model.predict([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = hindi_index_lookup[sampled_token_index]   
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    
    return decoded_sentence


In [ ]:
input_sentence = "and you know what its just the beginning"
output_sequence = get_predictions(input_sentence)
print("Generated Sequence:", output_sequence)
